In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%time
train = pd.read_csv("../input/tabular-playground-series-jan-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")
df=pd.concat([train,test],0).reset_index(drop=True)
df.sort_values(by=['id'],inplace=True)
df

In [ ]:
cols=df.drop(['id','target'],1).columns
df['mean']=df[cols].mean(axis=1)
df['min']=df[cols].min(axis=1)
df['max']=df[cols].max(axis=1)
df['std']=df[cols].std(axis=1)
df['kurt']=df[cols].kurt(axis=1)
df['mad']=df[cols].mad(axis=1)
df['median']=df[cols].median(axis=1)
df['skew']=df[cols].skew(axis=1)
df['sum']=df[cols].sum(axis=1)
df['sem']=df[cols].sem(axis=1)
df['idxmin']=df[cols].idxmin(axis=1).str.replace("cont","").astype(int)
df['idxmax']=df[cols].idxmax(axis=1).str.replace("cont","").astype(int)
df

In [ ]:
for c1 in cols:
    print(c1)
    for c2 in cols:
        if c1<c2:
            df[f'{c1}MOINS{c2}']=df[f'{c1}']-df[f'{c2}']
            df[f'{c1}PLUS{c2}']=df[f'{c1}']+df[f'{c2}']
            df[f'{c1}FOIS{c2}']=df[f'{c1}']*df[f'{c2}']
            df[f'{c1}DIV{c2}']=df[f'{c1}']/df[f'{c2}']
            


In [ ]:
# for c in df.drop(['id','target'],1).columns:
#     print(c)
#     for i in [-2,-1,1,2,3]:    
#         df[f'{c}_{i}']=df[f'{c}'].shift(i)


In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold
import lightgbm as lgb
import joblib
import time
target='target'
train=df[df['target'].notnull()]
test=df[~(df['target'].notnull())]
print(train.shape,test.shape)
y = train[target].copy()
X = train.drop([target,'id'],1)
features=X.columns
FOLDS=13
SEED=2021
from sklearn.metrics import mean_squared_error
oof=np.zeros(y.shape[0])
preds=np.zeros(test.shape[0])
skf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
for fold,(idxT,idxV) in enumerate( skf.split(X)):
    print("FOLD",fold)
    X_train,y_train=X.loc[idxT,features],y[idxT]
    X_test,y_test=X.loc[idxV,features],y[idxV]
    
    fit_params={"early_stopping_rounds":300, 
                "eval_metric" : ['rmse'],#'multi_error', 
                "eval_set" : [(X_test,y_test)],
                'eval_names': ['valid'],
                'verbose': 100,
                'feature_name': 'auto',
                'categorical_feature': 'auto' 
               }
    evals_result = {}
    clf = lgb.LGBMRegressor(num_leaves= 255, max_depth=-1, #15
                             seed=42, 
                             objective="regression",#num_class=y.nunique(), #len(unique),
#                              metric="custom",
                             silent=True, 
                             verbose=1,
                             num_threads=-1, 
                             n_estimators=100000,
                             colsample_bytree=0.7,
                             subsample=0.8,#class_weight='balanced',
                             learning_rate=0.01)        
    START=time.time()
    clf.fit(X_train, y_train, **fit_params)
    oof[idxV]=clf.predict(X_test)
    preds+=clf.predict(test[features])/FOLDS
    print("RMSE",mean_squared_error(y_test,oof[idxV]))
    END=time.time()
    print("time to train ",round(END-START,1),"secondes")
    feat_imp = pd.Series(clf.feature_importances_, index=X_train.columns)
    feat_imp=pd.DataFrame(feat_imp).reset_index()
    feat_imp.columns=['id','importance']
    feat_imp.sort_values('importance',ascending=False,inplace=True)
    joblib.dump(clf, f'{fold}.pkl')

    print(feat_imp[:50])

print("RMSE Global",mean_squared_error(y, oof))    
# [1646]	valid's rmse: 0.698171	valid's l2: 0.487443


In [ ]:
sample_submission=pd.read_csv("../input/tabular-playground-series-jan-2021/sample_submission.csv")
sample_submission['target'] = preds
sample_submission.to_csv('submission.csv', index=False)